In [32]:
import sklearn
from sklearn import utils, neighbors, ensemble, svm, metrics

In [2]:
# Load the libraries
import os
import imageio
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import seaborn as sns

In [3]:
#read_csv

labels=pd.read_csv("/Users/anaraquelpengelly/Desktop/MSC_health_data_science/term_2/machine_learning/project_malaria/Malaria_blood_image_classification/labels.csv")



In [9]:
#shuffle the df
random.seed(30)
labels = sklearn.utils.shuffle(labels)

In [10]:
labels.head()

,Unnamed: 0,0,infect_status
8090,58315,C39P4thinF_original_IMG_20150622_105335_cell_6...,1
26959,14862,C93P54ThinF_IMG_20150821_162641_cell_105.png,0
6785,51655,C182P143NThinF_IMG_20151201_172524_cell_204.png,1
26346,14177,C84P45ThinF_IMG_20150818_101146_cell_45.png,0
21850,9129,C227ThinF_IMG_20151112_135509_cell_173.png,0


In [37]:
labels.to_csv("/Users/anaraquelpengelly/Desktop/MSC_health_data_science/term_2/machine_learning/project_malaria/Malaria_blood_image_classification/shuffled_labels.csv")

## Splitting the training and test sets

Now we have a shuffled label df, we need to decide what the best way to split the training and test sets.
- 80% and 20% ? or something else? how do we determine the best way to do it?
- in any case we should have 50% of images that are infected and uninfected in the training set ( and the test set)

In [20]:
#sampleing labels for the training set:
training=labels.sample(frac=0.8, replace=False, random_state=30)

In [21]:
training.shape

(21946, 3)

In [22]:
training.head()

,Unnamed: 0,0,infect_status
8988,62154,C48P9thinF_IMG_20150721_161055_cell_195.png,1
26960,14863,C93P54ThinF_IMG_20150821_162641_cell_112.png,0
4945,43681,C170P131ThinF_IMG_20151119_120019_cell_244.png,1
4584,36096,C158P119ThinF_IMG_20151115_181136_cell_199.png,1
767,9586,C116P77ThinF_IMG_20150930_171558_cell_102.png,1


In [24]:
training["infect_status"].value_counts()

1    10973
0    10973
Name: infect_status, dtype: int64

In [26]:
training["0"].value_counts()

C85P46ThinF_IMG_20150820_110040_cell_185.png          1
C59P20thinF_IMG_20150803_113809_cell_39.png           1
C78P39ThinF_IMG_20150606_104106_cell_8.png            1
C39P4thinF_original_IMG_20150622_114122_cell_1.png    1
C161P122ThinF_IMG_20151116_101403_cell_101.png        1
                                                     ..
C144P105ThinF_IMG_20151015_160529_cell_170.png        1
C208ThinF_IMG_20151029_155827_cell_23.png             1
C133P94ThinF_IMG_20151004_154351_cell_94.png          1
C48P9thinF_IMG_20150721_161055_cell_195.png           1
C180P141NThinF_IMG_20151201_165659_cell_13.png        1
Name: 0, Length: 21946, dtype: int64

In [35]:
#sampleing lables for the test set: 
#Identify what values are in labels and not in training:
# %%timeit
diff=set(labels["0"]).difference(training["0"])
where_diff =labels["0"].isin(diff)
#slice labels and add create df
test=labels[where_diff]
test.shape

(5487, 3)

In [31]:
test.head()

,Unnamed: 0,0,infect_status
2868,20015,C132P93ThinF_IMG_20151004_152505_cell_135.png,1
22481,9842,C236ThinF_IMG_20151127_102635_cell_26.png,0
1506,11453,C119P80ThinF_IMG_20151002_123911_cell_139.png,1
4666,36762,C159P120ThinF_IMG_20151115_190642_cell_200.png,1
7459,52916,C184P145ThinF_IMG_20151203_104153_cell_164.png,1


In [33]:
test["infect_status"].value_counts()

1    2749
0    2738
Name: infect_status, dtype: int64

In [36]:
#now save both as csvs
path="/Users/anaraquelpengelly/Desktop/MSC_health_data_science/term_2/machine_learning/project_malaria/Malaria_blood_image_classification/"
training.to_csv((path+"training_labels.csv"))
test.to_csv((path+"test_labels.csv"))

In [41]:
#Just to check that thecsvs look fine 
a=pd.read_csv((path+"training_labels.csv"))
a.shape

(21946, 4)

In [42]:
b=pd.read_csv((path+"test_labels.csv"))
b.shape

(5487, 4)

In [43]:
c=pd.read_csv((path+"shuffled_labels.csv"))
c.shape

(27433, 4)

All looks good ecscept that we now have tw extra useless columns! 

Now we need to put all pictures in one folder and then create a function to grab from that folder the training and the test images.

## Split the training set in cross validation sets...

## Making a toy dataset label file

In [4]:
#randomly select 200 parasited and uninfected images:

toy=labels.sample(n=200, replace=False, random_state=30)



In [5]:
toy.shape

(200, 3)

In [6]:
toy["infect_status"].value_counts()

0    104
1     96
Name: infect_status, dtype: int64

This toy doesn't have the same number of uninfected and infected images, so I am going to try a different strategy:

In [43]:
grouped=labels.groupby("infect_status")
toy_df=grouped.apply(lambda x: x.sample(n=100, replace=False))
toy_df["infect_status"].value_counts()

1    100
0    100
Name: infect_status, dtype: int64

I will now check that they are unique images: 

In [44]:
print(len(toy_df.drop_duplicates()))

200


In [45]:
toy_df.head()

Unnamed: 0  \
infect_status                     
0             23734       11248   
              13904         197   
              21650        8900   
              24889       12535   
              24395       11985   

                                                                    0  \
infect_status                                                           
0             23734  C4thin_original_IMG_20150608_165540_cell_226.png   
              13904      C103P64ThinF_IMG_20150918_165016_cell_84.png   
              21650        C224ThinF_IMG_20151112_112834_cell_144.png   
              24889    C64P25N_ThinF_IMG_20150818_153517_cell_145.png   
              24395      C58P19thinF_IMG_20150802_122846_cell_100.png   

                     infect_status  
infect_status                       
0             23734              0  
              13904              0  
              21650              0  
              24889              0  
              24395              0

In [46]:
#just checking that the label is correct!
labels[labels["0"]=="C53P14thinF_IMG_20150726_114606_cell_11.png"]


,Unnamed: 0,0,infect_status
24018,11567,C53P14thinF_IMG_20150726_114606_cell_11.png,0


In [47]:
#shuffle: 
random.seed(30)
toy_df = sklearn.utils.shuffle(toy_df)

In [48]:
toy_df.head()

Unnamed: 0  \
infect_status                     
0             16932        3590   
              17201        3891   
              15066        1511   
1             3523        22306   
0             21050        8223   

                                                                 0  \
infect_status                                                        
0             16932  C145P106ThinF_IMG_20151016_154921_cell_52.png   
              17201  C149P110ThinF_IMG_20151115_114910_cell_17.png   
              15066  C120P81ThinF_IMG_20151002_130918_cell_169.png   
1             3523   C136P97ThinF_IMG_20151005_142437_cell_103.png   
0             21050     C215ThinF_IMG_20151106_133643_cell_204.png   

                     infect_status  
infect_status                       
0             16932              0  
              17201              0  
              15066              0  
1             3523               1  
0             21050              0

In [67]:
#now save df:
path="/Users/anaraquelpengelly/Desktop/MSC_health_data_science/term_2/machine_learning/project_malaria/Malaria_blood_image_classification/"
toy_df.to_csv(path+"toy_df.csv", index=False)

In [68]:
#test:
toy_df=pd.read_csv(path+"toy_df.csv")
toy_df.head()

,infect_status,0
0,0,C145P106ThinF_IMG_20151016_154921_cell_52.png
1,0,C149P110ThinF_IMG_20151115_114910_cell_17.png
2,0,C120P81ThinF_IMG_20151002_130918_cell_169.png
3,1,C136P97ThinF_IMG_20151005_142437_cell_103.png
4,0,C215ThinF_IMG_20151106_133643_cell_204.png


Need to see how to access the rownames of a df in pandas..

In [69]:
#now make the training and the test DFs
##sampleing labels for the training set:
training_toy=toy_df.sample(frac=0.8, replace=False, random_state=30)
##sampleing labels for the test set: 
diff=set(toy_df["0"]).difference(training_toy["0"])
where_diff =toy_df["0"].isin(diff)
#slice labels and add create df
test_toy=toy_df[where_diff]
test_toy.shape

(40, 2)

In [70]:
test_toy.head()

,infect_status,0
7,1,C126P87ThinF_IMG_20151004_104441_cell_118.png
11,0,C112P73ThinF_IMG_20150930_131516_cell_108.png
13,0,C95P56ThinF_IMG_20150821_172607_cell_82.png
15,1,C48P9thinF_IMG_20150721_162732_cell_6.png
18,0,C46P7ThinF_IMG_20151130_210123_cell_24.png


In [71]:
#save all the toys as csvs:
test_toy.to_csv(path+"test_toy.csv", index=False)
training_toy.to_csv(path+"training_toy.csv", index=False)